# Final script

## Loading libraries

In [ ]:
# Connect Google Drive on Google Colab

# from google.colab import drive

# drive.mount("/content/drive")

# Install detectron2 if you use Google Colab

# !python -m pip install pyyaml==5.1
# import sys, os, distutils.core
# !git clone 'https://github.com/facebookresearch/detectron2'
# dist = distutils.core.run_setup("./detectron2/setup.py")
# !python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
# sys.path.insert(0, os.path.abspath('./detectron2'))

## !python -m pip install "git+https://github.com/facebookresearch/detectron2.git" (pokud nefunguje přístup nahoře)

In [ ]:
import os
import torch, detectron2
import numpy as np
from PIL import Image
import cv2
import pandas as pd
import warnings

TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)
print("numpy:", np.__version__)
print("pandas:", pd.__version__)
warnings.filterwarnings("ignore")

In [ ]:
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor

setup_logger()

## Loading metadata and trained model

In [ ]:
train_images = "/home/james/Projects/Fenotypizace/data/annotations/Train/images/"
train_json = "/home/james/Projects/Fenotypizace/data/annotations/Train/result.json"
register_coco_instances("my_trainset", {}, train_json, train_images)
metadata = MetadataCatalog.get("my_trainset")

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(
    model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
)
cfg.DATALOADER.NUM_WORKERS = 8
cfg.MODEL.WEIGHTS = "/home/james/Projects/Fenotypizace/output/model_final.pth"
# cfg.MODEL.WEIGHTS = "/content/drive/Shareddrives/KIT ML2/Code/Final/1. Trays segmentation/output/model_final.pth" (Google Colab)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)

## Functions to customize the box naming based on position

In [ ]:
def num_of_columns(x_min):
    column = "Unknown_Column_"

    if 250 <= x_min <= 460:
        column = "1"

    elif 520 <= x_min <= 740:  # 413
        column = "2"

    elif 752 <= x_min <= 1030:  # 760
        column = "3"

    elif 1090 <= x_min <= 1300:
        column = "4"

    elif 1360 <= x_min <= 1625:
        column = "5"

    elif 1640 <= x_min <= 1900:
        column = "6"

    elif 1920 <= x_min <= 2170:
        column = "7"

    elif 2210 <= x_min <= 2470:
        column = "8"

    elif 2510 <= x_min <= 2900:
        column = "9"

    return column

In [ ]:
def num_of_rows(y_min):

    row = "Unknown_Row_"

    if 150 <= y_min <= 350:
        row = "1-"

    elif 380 <= y_min <= 580:
        row = "2-"

    elif 650 <= y_min <= 830:
        row = "3-"

    elif 920 <= y_min <= 1100:
        row = "4-"

    elif 1170 <= y_min <= 1330:
        row = "5-"

    elif 1400 <= y_min <= 1590:
        row = "6-"

    elif 1660 <= y_min <= 1840:
        row = "7-"

    elif 1880 <= y_min <= 2300:
        row = "8-"

    return row

## Function for the segmentation of the tray

In [ ]:
def fenotypizace(root, path):
    root_dir = os.scandir(root)
    df = pd.DataFrame(
        columns=["Sloupec_x_min", "Sloupec_x_max", "Radek_y_min", "Radek_y_max", "name"]
    )

    for img in root_dir:
        name = img.name[:-4]

        try:
            inp = cv2.imread(img.path)
            outputs = predictor(inp)
        except:
            print("Error soubor: " + name)

        # Takes the output/image from our model's prediction and makes it into a mask sheet, which is processed by the CPU
        masks = np.asarray(outputs["instances"].pred_masks.to("cpu"))
        num_of_iter = len(masks)

        for i in range(num_of_iter):
            item_mask = masks[i]  # Takes the mask of only one object from the image
            segmentation = np.where(item_mask == True)

            # Finding the boundary points of each box
            x_min = int(np.min(segmentation[1]))
            x_max = int(np.max(segmentation[1]))
            y_min = int(np.min(segmentation[0]))
            y_max = int(np.max(segmentation[0]))

            # Function to assign column and row based on box position
            column = num_of_columns(x_min)
            row = num_of_rows(y_min)
            box = row + column

            # Create a new folder to store a box based on location
            new_path = path + "/Tray_" + img.name[:2] + "/" + box + "/"
            isExist = os.path.exists(new_path)
            if not isExist:
                os.makedirs(new_path)

            # Cut box from image, Create mask and then cut box mask from image
            cropped = Image.fromarray(inp[y_min:y_max, x_min:x_max, :], mode="RGB")
            mask = Image.fromarray((item_mask * 255).astype("uint8"))
            cropped_mask = mask.crop((x_min, y_min, x_max, y_max))

            # Combining a box and a mask into one image, with the image in the front bordered by the mask = black
            new_fg_image = Image.new("RGB", cropped_mask.size)
            new_fg_image.paste(cropped, cropped_mask)

            # Storage of the cut-out box
            # If no column or row is recognized, the box is saved to the Unknown folder and a new row is added to the dataframe
            if column == "Unknown_Column_" or row == "Unknown_Row_":
                print("Chyba pri rozpoznani boxu")
                print(name + " - " + box)
                new_row = pd.DataFrame(
                    {
                        "Sloupec_x_min": [x_min],
                        "Sloupec_x_max": [x_max],
                        "Radek_y_min": [y_min],
                        "Radek_y_max": [y_max],
                        "name": [name + " - " + box],
                    }
                )
                df = pd.concat([df, new_row], ignore_index=True)
                cv2.imwrite(
                    new_path + name + "_" + box + ".png", np.array(new_fg_image)
                )

            else:
                cv2.imwrite(
                    new_path + name + "_" + box + ".png", np.array(new_fg_image)
                )  # .png
    df.to_csv(path + "/error_polohy.csv", index=False)

## Saving the results/boxes

In [ ]:
# # Local path
root = "/mnt/d/Záloha/Fenotypizace/data/raw/Obrázky/"
path = "/home/james/Projects/Fenotypizace/results/"

# # Local path - test
# root = "/home/james/Projects/Fenotypizace/data/test_segmentation/"
# path = "/home/james/Projects/Fenotypizace/results/segmations/"

# # Google Colab path
# root = "/content/drive/Shareddrives/Fenotypizace/EA2021-01Ohnoutkova/Full trays/"
# path = "/content/drive/Shareddrives/KIT ML/Fenotypizace - vzchazeni/Data/New_segmentations/"

fenotypizace(root, path)

print("All done")